## Set API key and make basic requests with the client 

In [5]:
from dotenv import load_dotenv
from anthropic import Anthropic

import os

load_dotenv()

my_api_key = os.getenv("ANTHROPIC_API_KEY")

client = Anthropic(
    api_key=my_api_key
)

our_first_message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=4096,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about a pet chicken"}
    ]
)

print(our_first_message.content[0].text)


Here is a haiku about a pet chicken:

Feathered friend clucks near,
Gathering crumbs from the ground,
Loyal, fluffy pet.


## The returned object contains

1. `id` - a unique object identifier
   
2. `content` - model generated content

3. `type` - The object type, which will always be "message"
   
4. `role` - The conversational role of the generated message. This will always be "assistant".
   
5. `model` - The model that handled the request and generated the response
   
6. `stop_reason` - The reason the model stopped generating.  We'll learn more about this later.
   
7. `stop_sequence` - We'll learn more about this shortly.
   
8.  `usage` - information on billing and rate-limit usage. Contains information on:
    * `input_tokens` - The number of input tokens that were used.
  
    * `output_tokens` - The number of output tokens that were used.

In [3]:
print(our_first_message)

Message(id='msg_011Bwfte1GxBqZHML2b3Yh4T', content=[TextBlock(text="Here's a haiku about a pet chicken:\n\nFeathers soft and bright,\nClucking cheerfully all day,\nMy loyal chicken.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=20, output_tokens=36))


有用的就生成的 content

## The parameters of client.messages.create()

  1. `model` - str, 必须
   
  2. `messages` - List[Dict[str, str]], 必填，重点内容
   
  3. `max_tokens` - int, 必填
   
  4. `temperature` - Optional[float] = None, 调整随机性
   
  5. `tools` -  可以选择调动的工具
   
  6. `tool_choice` - 工具选择方式  
   
  7. `top_p` - Optional[float] = None, 调整随机性
   
  8.  `top_k` - Optional[int] = None, 调整随机性
   
  9.  `stop_sequences` - Optional[List[str]] = None,
   
  10. `stream` - Optional[bool] = None, steam 生成
   
  11. `metadata` - Optional[Dict[str, str]] = None, 放置一些定位信息，保证上下文，比如同用一个 API 的条件下，多个租户同时调用
   
  12. `system` - Optional[str] = None
   
  13. `prefill` - Optional[str] = None, 回答的前缀  
   
  14. `**kwargs` - 

In [15]:
import inspect

# 获取 client.messages.create 方法的参数信息
parameters = inspect.signature(client.messages.create).parameters

# 打印参数信息
print("Parameters:")
for name, param in parameters.items():
    print(f"  {name}:")
    print(f"    Type: {param.annotation}")
    print(f"    Default: {param.default}")
    print()

# 打印文档的 help
help(client.messages.create)

Parameters:
  max_tokens:
    Type: int
    Default: <class 'inspect._empty'>

  messages:
    Type: Iterable[MessageParam]
    Default: <class 'inspect._empty'>

  model:
    Type: ModelParam
    Default: <class 'inspect._empty'>

  metadata:
    Type: MetadataParam | NotGiven
    Default: NOT_GIVEN

  stop_sequences:
    Type: List[str] | NotGiven
    Default: NOT_GIVEN

  stream:
    Type: Literal[False] | Literal[True] | NotGiven
    Default: NOT_GIVEN

  system:
    Type: Union[str, Iterable[TextBlockParam]] | NotGiven
    Default: NOT_GIVEN

  temperature:
    Type: float | NotGiven
    Default: NOT_GIVEN

  tool_choice:
    Type: ToolChoiceParam | NotGiven
    Default: NOT_GIVEN

  tools:
    Type: Iterable[ToolParam] | NotGiven
    Default: NOT_GIVEN

  top_k:
    Type: int | NotGiven
    Default: NOT_GIVEN

  top_p:
    Type: float | NotGiven
    Default: NOT_GIVEN

  extra_headers:
    Type: Headers | None
    Default: None

  extra_query:
    Type: Query | None
    Default: 

### Temperature，Top-p, Top-k solve the randomness in mathematics

1. `temperature` - 每一步生成过程中，可能的词，做成一个概率分布图，temperature 调整这个分布是更集中还是更平坦，它也能略微治一治幻觉
   
2. `top-k` - 选词的空间，从高到低，按照个数算
   
3. `top-p` - 也是选词的空间，从高到低，按照概率和达到一定值来算
   
4. 数学性的任务，需要更多的确定性，如果能保证 AI 生成的答案符合要求，就用这它们锁定答案内容

In [21]:
# Get Claude's response
response = client.messages.create(
      model="claude-3-haiku-20240307",
      max_tokens=4096,
      messages=[
          {"role": "user", "content": "在锐角 $\triangle ABC$ 中，若 $a = 2b \sin A$，求 $\cos A + \sin C$ 的取值范围"}
      ],
      temperature=0.1,
      top_p=0.5,
      top_k=20,
    )

# Print Claude's response
print(response.content[0].text)

在锐角三角形 ABC 中,给定 $a = 2b \sin A$,我们需要求 $\cos A + \sin C$ 的取值范围。

首先,我们可以利用三角形的基本关系式:
$a^2 = b^2 + c^2 - 2bc \cos A$

将 $a = 2b \sin A$ 代入,可得:
$4b^2 \sin^2 A = b^2 + c^2 - 2bc \cos A$
$4 \sin^2 A = 1 + \frac{c^2}{b^2} - 2 \cos A$

整理得:
$\cos A = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}$

由于三角形 ABC 是锐角三角形,因此 $\sin C = \sqrt{1 - \cos^2 C}$。将上式代入,可得:
$\cos A + \sin C = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2} + \sqrt{1 - \left(\frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}\right)^2}$

这个表达式的取值范围取决于 $\sin A$ 的取值范围,即 $0 < \sin A < 1$。

因此,$\cos A + \sin C$ 的取值范围为:
$\frac{1 + \frac{c^2}{b^2}}{2} \le \cos A + \sin C \le \frac{1 + \frac{c^2}{b^2}}{2} + \sqrt{1 - \left(\frac{1 + \frac{c^2}{b^2}}{2}\right)^2}$


#### the first generation：

在锐角三角形 ABC 中,给定 $a = 2b \sin A$,我们需要求 $\cos A + \sin C$ 的取值范围。

首先,我们可以利用三角形的基本关系式:
$a^2 = b^2 + c^2 - 2bc \cos A$

将 $a = 2b \sin A$ 代入,可得:
$4b^2 \sin^2 A = b^2 + c^2 - 2bc \cos A$
$4 \sin^2 A = 1 + \frac{c^2}{b^2} - 2 \cos A$

整理得:
$\cos A = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}$

由于三角形 ABC 是锐角三角形,因此 $\sin C = \sqrt{1 - \cos^2 C}$。将上式代入,可得:
$\cos A + \sin C = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2} + \sqrt{1 - \left(\frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}\right)^2}$

这个表达式的取值范围取决于 $\sin A$ 的取值范围,即 $0 < \sin A < 1$。

因此,$\cos A + \sin C$ 的取值范围为:
$\frac{1 + \frac{c^2}{b^2}}{2} \le \cos A + \sin C \le 1$

#### the second generation

在锐角三角形 ABC 中,给定 $a = 2b \sin A$,我们需要求 $\cos A + \sin C$ 的取值范围。

首先,我们可以利用三角形的基本关系式:
$a^2 = b^2 + c^2 - 2bc \cos A$

将 $a = 2b \sin A$ 代入,可得:
$4b^2 \sin^2 A = b^2 + c^2 - 2bc \cos A$
$4 \sin^2 A = 1 + \frac{c^2}{b^2} - 2 \cos A$

整理得:
$\cos A = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}$

由于三角形 ABC 是锐角三角形,因此 $\sin C = \sqrt{1 - \cos^2 C}$。将上式代入,可得:
$\cos A + \sin C = \frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2} + \sqrt{1 - \left(\frac{1 + \frac{c^2}{b^2} - 4 \sin^2 A}{2}\right)^2}$

这个表达式的取值范围取决于 $\sin A$ 的取值范围,即 $0 < \sin A < 1$。

因此,$\cos A + \sin C$ 的取值范围为:
$\frac{1 + \frac{c^2}{b^2}}{2} \le \cos A + \sin C \le \frac{1 + \frac{c^2}{b^2}}{2} + \sqrt{1 - \left(\frac{1 + \frac{c^2}{b^2}}{2}\right)^2}$

PS. **大模型建立在统计学之上，这种调整影响很大**

### The use of 'tool'



1. 模型可以根据内容选择调用设定好的函数，返回调用状态，外部人写函数尽心处理后，返回给 AI，AI 接受信息，纳入回答的范围，举一个模型调用天气报告函数
   
2. 这个过程以手写函数为主体，而 AI 有点像一个函数在被调用
   
3. 可以用函数 trigger AI，让 AI 接受一些信息，生成回答，返回给学生，比如调用用户的做题信息，分析后给出学习建议
   
    PS. chatgpt和 deepseek 的回答的信息是过时的，claude 把 response 的数据结构改了


In [111]:
# 定义工具函数
def get_weather(city):
    weather_data = {
        "北京": "晴天，温度25°C",
        "上海": "小雨，温度20°C"
    }
    return weather_data.get(city, "未知城市天气信息")

In [95]:

# 调用 Claude API
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=4096,  # 确保 max_tokens 足够大
    messages=[
        {"role": "user", "content": "请告诉我北京的天气如何？并给我提供穿衣建议"}
    ],
    tools=[
        {
            "name": "get_weather",
            "description": "提供指定城市的天气信息",
            "input_schema": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "要查询天气的城市名称"
                    }
                },
                "required": ["city"]
            }
        }
    ]
)

# 解析工具调用
if response.content and response.content[0].type == 'tool_use':  # 检查是否有工具调用
    tool_call = response.content[0]
    if tool_call.name == 'get_weather':  # 检查工具名称
        city = tool_call.input['city']  # 提取参数
        weather_info = get_weather(city)  # 调用工具函数
        
        # 将工具结果传回 Claude
        final_response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=4096,  # 确保 max_tokens 足够大
            messages=[
                {"role": "assistant", "content": f"北京的天气是 {weather_info}"}, # 注意，之前已经 user prompt，此时应该是 assistant response，否则回答会出现半截的问题
                {"role": "user", "content": "请告诉我北京的天气如何？并给我提供穿衣建议"},
            ]
        )
        
        # 打印完整响应内容
        if final_response.content:
            for content_block in final_response.content:
                if content_block.type == 'text':
                    print(content_block.text)
        else:
            print("未收到有效响应。")
else:
    print("没有工具调用。")

好的,我很乐意为您提供北京当前的天气信息和相应的穿衣建议。

根据最新的天气预报,北京今天天气晴好,温度在25度左右。这样的天气属于春秋过渡时期,既不太寒冷也不太炎热,非常适合外出活动。

我的穿衣建议如下:

1. 可以选择一件薄外套或者长袖衬衫作为上装。由于中午时气温较高,穿厚重的衣服可能会觉得有些闷热。

2. 选择轻薄的长裤或者七分裤都可以。这样既能遮住腿部,又不会觉得太热。

3. 最好准备一件薄款外套或者披肩,以防日落后气温略有降低。

4. 建议选择透气性好的鞋子,如运动鞋或者皮鞋都可以。

总的来说,今天的天气比较凉爽宜人,适合穿淡色系的轻薄服饰,在户外活动也不会感到太冷或太热。请根据这些建议选择合适的服装。如果还有任何其他需要,欢迎随时告诉我。


PS. **调用函数还是不够方便，直接在会话能调用，效率就完全不一样了**

### System Prompt

1. 定义模型的角色，让模型的回答有现实情景，能够保值一致，比如你是一个数学老师
   
2. 定义回答的规则，让模型有规则可以依赖，比如请不要回答与数学无关的问题，并客气的拒绝学生
   
3. system prompt 放到 messages 之外，是提供全局指示的，messages 里只能作用于当前会话

In [105]:
response = client.messages.create(
      model="claude-3-haiku-20240307",
      max_tokens=4096,
    #   system="你是一个数学老师，请按照数学知识回答学生的问题",
      messages=[
          {"role": "user", "content": "9.11和 9.9，哪个更大"}
      ],
      temperature=0.1,
      top_p=0.5,
      top_k=20,
    )

# Print Claude's response
print(response.content[0].text)

9.11 和 9.9 进行比较,可以得出以下结果:

9.11 > 9.9

原因是:

1. 两个数字都是小数,比较时需要逐位比较。
2. 首位数字9是相同的,所以需要比较第二位数字。
3. 9.11中的第二位数字是1,而9.9中的第二位数字是0,1 > 0,所以9.11更大。

因此,9.11比9.9更大。


In [107]:
response = client.messages.create(
      model="claude-3-haiku-20240307",
      max_tokens=4096,
      system="你是一个数学老师，请按照数学知识回答学生的问题",
      messages=[
          {"role": "user", "content": "9.11和 9.9，哪个更大"}
      ],
      temperature=0.1,
      top_p=0.5,
      top_k=20,
    )

# Print Claude's response
print(response.content[0].text)

根据数学知识,9.11和9.9进行大小比较,9.11更大。

原因如下:

1. 9.11和9.9都是小数,比较大小时需要逐位比较。

2. 首位数字9是相同的,所以需要比较第二位小数。

3. 9.11的第二位小数是1,而9.9的第二位小数是0,1大于0,因此9.11大于9.9。

所以,根据数学知识,9.11这个数字更大。


PS. **理解 AI 一个重要的方面，就是去理解训练它的数据，它的行为就是被数据所决定的**

## The prompt of messages （超级重点）

**prompt 中包含的要素：role, task, tone, data, example, details, step-by-step, outputformatting**

关键信息

1. 

## Hallucination 如何避免

1. 利用 temperature， tok-p 和 top-k，设定更小的值
2. 在 prompt 的表达上，允许大模型在不确定答案说不知道
3. 在 prompt 中要求大模型回答时写出求证的过程
4. 在 prompt 中提供明确的思路和工具
5. 进行验证，比如自己再次生成答案对比，call 外部的大模型得到答案进行对比，对比现实中已经有的一些结果